# Framework Demo: Sequential Questions Solving



## 📖 Overview



This notebook demonstrates the **Adaptive LLM-Symbolic Reasoning Framework** solving **multiple sequential reasoning problems** in a single prompt.



### Key Capabilities



- **Multi-Question Parsing:** Automatically separates and identifies individual questions

- **Heterogeneous Reasoning:** Applies different solvers to different problem types

- **Parallel Execution:** Processes independent questions concurrently when possible

- **Individual Evaluation:** Provides per-question accuracy metrics



### Problem Types in This Demo



<div style="display: flex; gap: 10px; margin-top: 15px;">

<div style="flex: 1; background-color: #e8f8f5; padding: 12px; border-radius: 5px; border-left: 4px solid #16a085;">

<b>Q1: Constraint Satisfaction</b><br>

<small>Spatial reasoning with constraints</small>

</div>

<div style="flex: 1; background-color: #ebf5fb; padding: 12px; border-radius: 5px; border-left: 4px solid #3498db;">

<b>Q2: Deductive Reasoning</b><br>

<small>Logical inference from facts and rules</small>

</div>

<div style="flex: 1; background-color: #f4ecf7; padding: 12px; border-radius: 5px; border-left: 4px solid #8e44ad;">

<b>Q3: First-Order Logic</b><br>

<small>Formal logic with quantifiers</small>

</div>

</div>



### Dataset



**concat dataset:** Combines problems from Trec, LogDed7, ProofWriter, ProntoQA, and FOLIO datasets. Each sample contains 3 questions of varying types.


### Processing Workflow
<img src="./main_pic.png"  width="800">

## Step 1: Load Multi-Question Sample

Load a sample containing multiple reasoning problems from the concat dataset.

In [1]:
import json
import yaml
import warnings
warnings.filterwarnings("ignore")

# Load configuration
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Load a sample from concat dataset
data_path = 'data/concat/concat.json'
with open(data_path, 'r') as f:
    concat_data = json.load(f)

# Select the first sample (you can modify the index to try other samples)
sample = concat_data[5]
if not isinstance(sample['answer'], list):
    sample['answer'] = [sample['answer']]

print(f"Sample ID: {sample['id']}")
print(f"Problem Type: {sample['type']}")
print(f"Number of Questions: {sample['num_problems']}")
print(f"Original IDs: {sample['original_ids']}")
print(f"\nProblem Description:")
print(sample['problem'])
print(f"\nGround Truth Answers: {sample['answer']}")

Sample ID: multi_problem_6
Problem Type: ['Constraint Satisfaction', 'Deductive Reasoning', 'First-order Logic']
Number of Questions: 3
Original IDs: ['CSP_215', 'lp_ProofWriter_RelNoneg-OWA-D5-637_Q19', 'FOL_FOLIO_dev_168']

Problem Description:
Answer the following questions one by one.

Q1: STATEMENT:
On a branch, there are seven birds: a falcon, a crow, a hawk, a hummingbird, a blue jay, a robin, and a raven. The blue jay is to the right of the robin. The hawk is to the left of the hummingbird. The robin is the second from the right. The falcon is the third from the left. The crow is to the right of the hummingbird. The raven is the second from the left.

Which of the following is true?

A. The falcon is the second from the right.
B. The crow is the second from the right.
C. The hawk is the second from the right.
D. The hummingbird is the second from the right.
E. The blue jay is the second from the right.
F. The robin is the second from the right.
G. The raven is the second from t

## Step 2: Execute Reasoning with Router

The **router** automatically:
- Parses and separates individual questions
- Identifies problem type for each question
- Creates a unified execution plan (DAG)
- Routes each question to the appropriate solver
- Executes reasoning and collects results

In [3]:
from agents.generation.api import AzureOpenAIGenerator
from agents.meta_agents.planner import TracePersister, Planner

# Initialize Azure OpenAI (using gpt-4o-azure config from config.yaml)
azure_config = config['api_config']['gpt-4o-azure']
llm = AzureOpenAIGenerator(
    model_name=azure_config['model_name'],
    api_key=azure_config['api_key'],
    model_version=azure_config['openai_api_version'],
    azure_endpoint=azure_config['azure_endpoint']
)

# Initialize Carnap router
router = Planner(generator=llm)
print("✓ Carnap router initialized with Azure OpenAI (gpt-4o)")

problem_types = []
try:
    # Router creates execution plan
    plan, memory, problem_ids = router(sample)
    plan = plan[0]
    
    # Execute reasoning plan
    print("\nExecuting reasoning...")
    plan.execute(memory, TracePersister())
    
    # Extract results for each sub-question
    predictions = []
    for pid in problem_ids:
        result = memory.read(f"result_{pid}") or memory.read(f"result_no_problem") or {}
        predictions.append(result.get('parsed_answer', None))
    
    print("✓ Reasoning completed")
    
    # Evaluate results
    is_all_correct = all(pred == gt for pred, gt in zip(predictions, sample['answer']))
    is_partial_correct = [pred == gt for pred, gt in zip(predictions, sample['answer'])]


    # Get predicted problem types for each sub-question
    problem_types = [memory.read(f'problem_type_{pid}') for pid in problem_ids]
    print(f"\nPredicted problem type(s): {problem_types}")
    print(f"Agents used: {list(plan.agents.keys())}")
    
    # Display plan structure
    print(f"\nPlan Structure (DAG):")
    for edge in plan.edges:
        print(f"  {edge.source} -> {edge.target}")
    
    print("\n" + "=" * 60)
    print("Result Comparison")
    print("=" * 60)
    for i, (pred, gt, correct) in enumerate(zip(predictions, sample['answer'], is_partial_correct)):
        status = '✅' if correct else '❌'
        print(f"Q{i+1}: Ground Truth: {gt}, Prediction: {pred} {status}")
    print(f"\nOverall Evaluation: {'✅ All Correct' if is_all_correct else '❌ Some Incorrect'}")
    print(f"Accuracy: {sum(is_partial_correct)}/{len(is_partial_correct)}")
    print("=" * 60)
    
except Exception as e:
    import traceback
    
    # Capture full traceback as a formatted string
    traceback_str = ''.join(traceback.format_exception(type(e), e, e.__traceback__))
    
    # Print error information
    print("\n" + "=" * 60)
    print("❌ ERROR OCCURRED")
    print("=" * 60)
    print(f"Error Type: {type(e).__name__}")
    print(f"Error Message: {str(e)}")
    print(f"\nPredicted Problem Types: {problem_types}")
    
    # Try to show plan information if available
    try:
        print(f"\nAgents in Plan: {list(plan.agents.keys())}")
        print(f"\nPlan DAG:")
        for edge in plan.edges:
            print(f"  {edge.source} -> {edge.target}")
    except:
        pass
    
    print(f"\nFull Traceback:")
    print(traceback_str)
    print("=" * 60)

2025-10-27 19:58:19,811 | INFO | Registered agent '<PLAN_START>'
2025-10-27 19:58:19,812 | INFO | Registered agent '<PLAN_END>'
2025-10-27 19:58:19,812 | INFO | Registered agent 'lp_solver'
2025-10-27 19:58:19,812 | INFO | Registered agent 'fol_solver'
2025-10-27 19:58:19,813 | INFO | Registered agent 'csp_solver'
2025-10-27 19:58:19,813 | INFO | Registered agent 'smt_solver'
2025-10-27 19:58:19,813 | INFO | Registered agent 'ilp_solver'
2025-10-27 19:58:19,814 | INFO | Registered agent 'epistemic_solver'
2025-10-27 19:58:19,814 | INFO | Registered agent 'risk_solver'
2025-10-27 19:58:19,815 | INFO | Registered agent 'compositional_solver'
2025-10-27 19:58:19,815 | INFO | Registered agent 'causal_solver'


✓ Carnap router initialized with Azure OpenAI (gpt-4o)


2025-10-27 19:58:23,894 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
2025-10-27 19:58:23,896 | INFO | Scratchpad WRITE problem_type_ques_1=CSP (ttl=None)
2025-10-27 19:58:23,896 | INFO | Scratchpad WRITE context_ques_1=On a branch, there are seven birds: a falcon, a crow, a hawk (ttl=None)
2025-10-27 19:58:23,897 | INFO | Scratchpad WRITE question_ques_1=Which of the following is true? (ttl=None)
2025-10-27 19:58:23,897 | INFO | Scratchpad WRITE options_ques_1=A. The falcon is the second from the right.
B. The crow is t (ttl=None)
2025-10-27 19:58:23,897 | INFO | Scratchpad WRITE csp_input_ques_1={'problem': 'On a branch, there are seven birds: a falcon, a (ttl=None)
2025-10-27 19:58:23,898 | INFO | Scratchpad WRITE description_ques_1=For a Constraint Satisfaction Problem (CSP), find a variable (ttl=None)
2025-10-27 19:58:23,898 | INFO | Scratchpad WRITE problem_type_ques_


Executing reasoning...


2025-10-27 19:58:26,894 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
2025-10-27 19:58:26,912 | INFO | Scratchpad WRITE intermediate_form_fol_solver=Predicates:
NaziPolitician(x) ::: x is a Nazi German politic (ttl=None)
2025-10-27 19:58:26,912 | INFO | Attempt 1/1
2025-10-27 19:58:26,918 | INFO | Scratchpad WRITE critique_outputs_fol_solver=True (ttl=None)
2025-10-27 19:58:26,918 | INFO | Prover9 solver ran successfully.
2025-10-27 19:58:27,297 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
2025-10-27 19:58:27,299 | INFO | Scratchpad WRITE solver_label_fol_solver=A (ttl=None)
2025-10-27 19:58:27,299 | INFO | Scratchpad WRITE result_ques_3={'ori_answer': 'True', 'parsed_answer': 'A', 'intermediate_f (ttl=None)
2025-10-27 19:58:27,299 | INFO | Trace saved: fol_sol

✓ Reasoning completed

Predicted problem type(s): ['CSP', 'LP', 'FOL']
Agents used: ['<PLAN_START>', 'ques_3', 'ques_2', 'ques_1', 'fol_solver', 'lp_solver', 'csp_solver', '<PLAN_END>']

Plan Structure (DAG):
  <PLAN_START> -> ques_3
  <PLAN_START> -> ques_2
  <PLAN_START> -> ques_1
  ques_1 -> csp_solver
  csp_solver -> <PLAN_END>
  ques_2 -> lp_solver
  lp_solver -> <PLAN_END>
  ques_3 -> fol_solver
  fol_solver -> <PLAN_END>

Result Comparison
Q1: Ground Truth: F, Prediction: F ✅
Q2: Ground Truth: C, Prediction: C ✅
Q3: Ground Truth: A, Prediction: A ✅

Overall Evaluation: ✅ All Correct
Accuracy: 3/3
